In [404]:
import pandas as pd
import re
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [405]:
import json
import folium

folium.__version__ == '0.5.0'

True

In [406]:
df = pd.read_csv('eurostat.tsv', sep='\t')
df.columns = ['Region', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016']
df = df[['Region', '2016']]
df['Region'] = df['Region'].apply(lambda x: x.split(',', 1)[0])
df.drop(df.tail(2).index, inplace=True)
df.drop(df.head(6).index, inplace=True)

df['2016'] = df['2016'].apply(lambda x: re.sub('[bui ]', '', x)).astype('float')
df['Region'] = df['Region'].replace('Germany (until 1990 former territory of the FRG)', 'Germany')
df.head()

Region  2016
6          Belgium   4.7
7         Bulgaria   4.5
8   Czech Republic   2.6
9          Denmark   4.3
10         Germany   2.9

In [407]:
df.describe()

2016
count  31.000000
mean    5.364516
std     2.652112
min     2.500000
25%     3.650000
50%     4.800000
75%     6.150000
max    14.100000

In [408]:
m_eu = folium.Map([54, 15], tiles='cartodbpositron', zoom_start=4)

In [409]:
js = r'topojson/europe.topojson.json'
js_data = json.load(open(js))
js_data

{'arcs': [[[9439, 2245],
   [-14, 6],
   [-31, 9],
   [-31, 11],
   [-12, 5],
   [-11, 5],
   [-2, 7],
   [0, 8],
   [-10, 25],
   [-5, 9],
   [-2, 1],
   [-1, -2],
   [-3, -2],
   [-16, 7],
   [-1, 2],
   [1, 4],
   [-2, 11],
   [-2, 6],
   [-7, 14],
   [-9, 11],
   [-3, 1],
   [-3, 3],
   [-3, 11],
   [-3, 14],
   [-2, 17],
   [-2, 3],
   [-8, 2]],
  [[9257, 2433], [0, 5], [-3, 12], [-2, 2]],
  [[9252, 2452],
   [3, 2],
   [8, 3],
   [3, 0],
   [5, -2],
   [7, 4],
   [3, 4],
   [4, 6],
   [5, 1],
   [3, -3],
   [24, -37],
   [7, -17],
   [2, -2],
   [17, -11],
   [5, 2],
   [12, 12],
   [17, 8],
   [3, 0],
   [8, -5],
   [3, -3],
   [1, -27],
   [4, -21],
   [18, -18],
   [2, -6],
   [0, -2],
   [-5, -8],
   [0, -10],
   [18, -55],
   [10, -22]],
  [[9313, 2757], [-1, -2], [-3, 3], [-1, 3], [2, 2], [3, -4], [0, -2]],
  [[9288, 2771],
   [-3, -1],
   [-2, 1],
   [-3, 3],
   [-1, 3],
   [1, 5],
   [1, 2],
   [5, -1],
   [3, -5],
   [0, -5],
   [-1, -2]],
  [[9760, 2963],
   [4, -3],
  

In [410]:
regions = df['Region'].tolist()
countries = pd.DataFrame(
    {'Region': 
     [x['properties']['NAME'] for x in js_data['objects']['europe']['geometries'] if x['properties']['NAME'] in regions]}
)
countries

Region
0         Bulgaria
1           Cyprus
2          Denmark
3          Ireland
4          Estonia
5          Austria
6   Czech Republic
7          Finland
8           France
9          Germany
10          Greece
11         Croatia
12         Hungary
13         Iceland
14           Italy
15          Latvia
16       Lithuania
17        Slovakia
18           Malta
19         Belgium
20      Luxembourg
21     Netherlands
22          Norway
23          Poland
24        Portugal
25         Romania
26        Slovenia
27           Spain
28          Sweden
29          Turkey
30  United Kingdom

In [411]:
data_map = countries.merge(df, on='Region', how='outer')
data_map

Region  2016
0         Bulgaria   4.5
1           Cyprus   8.6
2          Denmark   4.3
3          Ireland   5.1
4          Estonia   4.8
5          Austria   4.1
6   Czech Republic   2.6
7          Finland   5.8
8           France   6.1
9          Germany   2.9
10          Greece  14.1
11         Croatia   7.5
12         Hungary   3.1
13         Iceland   2.5
14           Italy   6.6
15          Latvia   6.6
16       Lithuania   5.3
17        Slovakia   6.2
18           Malta   2.8
19         Belgium   4.7
20      Luxembourg   4.0
21     Netherlands   4.2
22          Norway   3.3
23          Poland   3.8
24        Portugal   7.3
25         Romania   3.5
26        Slovenia   5.0
27           Spain  12.8
28          Sweden   5.0
29          Turkey   5.9
30  United Kingdom   3.3

In [412]:
eu = js_data['objects']['europe']['geometries']
regs = data_map['Region'].tolist()
new_eu = [x for x in eu if x['properties']['NAME'] in regs]
js_data['objects']['europe']['geometries'] = new_eu

In [413]:
def color_country(name):
    if(name in regs):
        return '#996DAD'
    else:
        return '#727272'

In [414]:
folium.TopoJson(open(js), 'objects.europe',
               style_function=lambda feature: {
                'fillColor': color_country(feature['properties']['NAME']),
                'color': 'black',
                'weight': 2   
                }).add_to(m_eu)

In [415]:
m_eu

In [416]:
m_eu.choropleth(geo_data=js_data, data=data_map,
         columns=['Region', '2016'],topojson = 'objects.europe',
         key_on='feature.properties.NAME',
         fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
         legend_name='test', highlight = True)

In [417]:
m_eu